### Analysis V2


In [ ]:
import dask.dataframe as dd
import dask_geopandas as dg
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely.wkt
import utilities
from calitp import *
from shared_utils import geography_utils, utils

### Load in Provider Maps

In [ ]:
# Simple Version
verizon_simple, att_simple, tmobile_simple = utilities.simplify_geometry_all_providers()

In [ ]:
# Geometries are true across each gdf
verizon_simple.geometry.is_valid.all(), att_simple.geometry.is_valid.all(), tmobile_simple.geometry.is_valid.all(),

In [ ]:
# verizon_og =utilities.load_verizon()
# att_og = utilities.load_att()
# tmobile_og = utilities.load_tmobile()

### Load in Routes 

In [ ]:
unique_routes = utilities.load_unique_routes_df()

In [ ]:
len(unique_routes)

In [ ]:
# Ensure CRS are the same
unique_routes.crs == att_simple.crs == tmobile_simple.crs == verizon_simple.crs

### Overlay

In [ ]:
# Routes from analysis v1 that didn't make sense
routes_wanted = [
    "Edmund D. Edelman Children’s Court Shuttle",
    "Oakland - San Francisco",
    "Foster City - Hillsdale Mall",
    "24 hour service daily",
    "Mammoth Lakes HWY 120E/395",
    "Avocado Heights/Bassett/West Valinda Shuttle",
    "Long Beach/San Pedro",
]

In [ ]:
unique_routes_subset = unique_routes[unique_routes["route_name"].isin(routes_wanted)]

In [ ]:
len(unique_routes_subset)

In [ ]:
intersect_att = utilities.comparison(unique_routes_subset, att)

In [ ]:
intersect_att

In [ ]:
# Error message: /tmp/ipykernel_1424/566793742.py:1: UserWarning: `keep_geom_type=True` in overlay
# resulted in 69 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
intersect_att1 = gpd.overlay(
    att, unique_routes_subset, how="intersection", keep_geom_type=False
)

In [ ]:
intersect_tmobile = gpd.overlay(
    tmobile, unique_routes_subset, how="intersection", keep_geom_type=False
)

In [ ]:
intersect_tmobile = intersect_tmobile.assign(
    route_length=intersect_tmobile.geometry.to_crs(geography_utils.CA_StatePlane).length
)

In [ ]:
intersect_tmobile

In [ ]:
# TypeError: `keep_geom_type` does not support GeometryCollection.
intersect_verizon = gpd.overlay(
    verizon, unique_routes_subset, how="intersection", keep_geom_type=False
)

In [ ]:
intersect_verizon

In [ ]:
def overlay_single_routes(
    provider: gpd.GeoDataFrame, routes: gpd.GeoDataFrame, provider_name: str
):
    """
    For any provider or routes that may cause an issue, 
    use this function to pass and print out the names of 
    problematic routes. 
    """

    # Empty dataframe to hold results
    all_intersected_routes = pd.DataFrame()

    # List of long route names
    unique_routes_list = routes.long_route_name.unique().tolist()

    # Intersect route by route, skipping those that don't work
    for i in unique_routes_list:
        filtered = routes[routes.long_route_name == i].reset_index(drop=True)
        try:
            intersected = gpd.overlay(
                provider, filtered, how="intersection", keep_geom_type=False
            )
            all_intersected_routes = pd.concat(
                [all_intersected_routes, intersected], axis=0
            )
        except:
            pass
            print(f"{i}")

    utils.geoparquet_gcs_export(intersected_routes, utilities.GCS_FILE_PATH, f"{provider_name}_intersected_routes")

    return all_intersected_routes

In [2]:
att_o = utilities.comparison(att_simple, unique_routes)

NameError: name 'utilities' is not defined

In [ ]:
# utils.geoparquet_gcs_export(verizon_o, utilities.GCS_FILE_PATH, "verizon_simplified_15")

In [ ]:
# Routes that didn't work when using verizon simplified 

In [ ]:
"""def split_routes():
    unique_routes = utilities.load_unique_routes_df()
    # Split unique_routes to 4 different ones
    # https://stackoverflow.com/questions/17315737/split-a-large-pandas-dataframe
    df_split = np.array_split(unique_routes, 4)
    
    df1 = df_split[0]
    df2 = df_split[1]
    df3 = df_split[2]
    df4 = df_split[3]
    
    return df1,df2,df3,df4
"""